### Konstantina Andronikou

## Comparison with notebook with_conjuction

## This noteboook is implementing a sentence without a conjuction

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
# To remove all warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Someone killed Mary.")
output

{'verbs': [{'verb': 'killed',
   'description': '[ARG0: Someone] [V: killed] [ARG1: Mary] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['Someone', 'killed', 'Mary', '.']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone killed Mary."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'killed',
     'description': '[ARG0: Someone] [V: killed] [ARG1: Mary] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['Someone', 'killed', 'Mary', '.']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg1_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name'],meta['last_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('Someone killed {first_name} {last_name}.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'last_name': 'Johnson', 'first_name': 'Ian'}, {'last_name': 'Wright', 'first_name': 'Kathryn'}, {'last_name': 'Brooks', 'first_name': 'Michelle'}, {'last_name': 'Butler', 'first_name': 'Janet'}, {'last_name': 'Wood', 'first_name': 'Marilyn'}, {'last_name': 'Hughes', 'first_name': 'Betty'}, {'last_name': 'Smith', 'first_name': 'Alexander'}, {'last_name': 'King', 'first_name': 'Sharon'}, {'last_name': 'Hall', 'first_name': 'Marilyn'}, {'last_name': 'Reed', 'first_name': 'Annie'}, {'last_name': 'Smith', 'first_name': 'Tim'}, {'last_name': 'Roberts', 'first_name': 'Harold'}, {'last_name': 'Hall', 'first_name': 'Lauren'}, {'last_name': 'James', 'first_name': 'Ruth'}, {'last_name': 'Reynolds', 'first_name': 'Kevin'}, {'last_name': 'Smith', 'first_name': 'Keith'}, {'last_name': 'Reed', 'first_name': 'Brian'}, {'last_name': 'Sullivan', 'first_name': 'Paul'}, {'last_name': 'Harris', 'first_name': 'George'}, {'last_name': 'White', 'first_name': 'Amy'}, {'last_name': 'James', 'first_name':

In [12]:
with open('dataset/Without_conjuction.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/predictions_without_con.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    6 (0.6%)

Example fails:
[ARG0: Someone] [V: killed] [ARG1: Amy] [ARGM-TMP: Stone] .
----
[ARG0: Someone] [V: killed] [ARG1: Frederick] [ARGM-TMP: Stone] .
----
[ARG0: Someone] [V: killed] [ARG1: Frank] [ARGM-TMP: Henderson] .
----
['Someone', 'killed', 'Ian', 'Johnson', '.'] [ True]
['Someone', 'killed', 'Kathryn', 'Wright', '.'] [ True]
['Someone', 'killed', 'Michelle', 'Brooks', '.'] [ True]
['Someone', 'killed', 'Janet', 'Butler', '.'] [ True]
['Someone', 'killed', 'Marilyn', 'Wood', '.'] [ True]
['Someone', 'killed', 'Betty', 'Hughes', '.'] [ True]
['Someone', 'killed', 'Alexander', 'Smith', '.'] [ True]
['Someone', 'killed', 'Sharon', 'King', '.'] [ True]
['Someone', 'killed', 'Marilyn', 'Hall', '.'] [ True]
['Someone', 'killed', 'Annie', 'Reed', '.'] [ True]
['Someone', 'killed', 'Tim', 'Smith', '.'] [ True]
['Someone', 'killed', 'Harold', 'Roberts', '.'] [ True]
['Someone', 'killed', 'Lauren', 'Hall', '.'] [ True]


In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_without_con.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 